In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
import random
import zlib
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
def generate_random_dataset(n_samples, d):
    X = np.random.rand(n_samples, d)  # Random features between 0 and 1
    y = np.random.randint(2, size=n_samples)  # Random binary labels (0 or 1)
    return X, y

# Question 6.1 a

In [ ]:
#number of points have to memorize is average over all points that need to be kept with 1-NN
#n_full/n_average approaches 2 as d increases based on counting function theorem, so that n_full cn be reduced to n_full/2bits to prove efficient generalization and compression

#n_full = 2**D

In [ ]:
def condensed_dataset_knn(X, y):
    C_X = np.array([X[0]])
    C_y = np.array([y[0]])
    KNN = knn(n_neighbors=1)

    for i in range(1, len(X)):
        KNN.fit(C_X, C_y)
        if KNN.predict([X[i]])[0] != y[i]:
            C_X = np.append(C_X, [X[i]], axis=0)
            C_y = np.append(C_y, [y[i]])

    return C_X, C_y


In [ ]:
np.random.seed(42)
dimensionalities = [2, 4, 8]
functions_per_dimension = {2: 16, 4: 32, 8: 64}

for d, num_functions in functions_per_dimension.items():
    n_full = 2 ** d  # Total possible configurations
    memorization_counts = []

    for _ in range(num_functions):
        X, y = generate_random_dataset(n_full, d)  # Generate dataset for each function
        _, condensed_y = condensed_dataset_knn(X, y)  # Apply CNN
        memorization_counts.append(len(condensed_y))  # Record the number of points kept
    n_avg = np.mean(memorization_counts)
    print(f"d={d}: n_full={2**d}, Avg. req. points for memorization n_avg={n_avg:.2f}, n_full/n_avg={(2**d)/n_avg}")

d=2: n_full=4, Avg. req. points for memorization n_avg=2.19, n_full/n_avg=1.8285714285714285
d=4: n_full=16, Avg. req. points for memorization n_avg=8.50, n_full/n_avg=1.8823529411764706
d=8: n_full=256, Avg. req. points for memorization n_avg=129.39, n_full/n_avg=1.9785050114720444


# Question 6.1 b

In [ ]:
def generate_random_dataset_multi(n_samples, n_features, n_classes=3):
    X = np.random.rand(n_samples, n_features)  # Random features
    y = np.random.randint(0, n_classes, size=n_samples)  # Random labels for multiple classes
    return X, y

In [ ]:
dimensionalities = [2, 4, 8]
n_classes = 3
functions_per_dim = {2: 27, 4: 54, 8: 108}

for d in dimensionalities:
    all_memorization_sizes = []

    for func in range(functions_per_dim[d]):
        X, y = generate_random_dataset_multi(3**d, d, n_classes)
        # Apply the condensed KNN algorithm
        condensed_X, condensed_y = condensed_dataset_knn(X, y)
        # Store the size of the condensed dataset
        all_memorization_sizes.append(len(condensed_y))

    avg_mem_size = np.mean(all_memorization_sizes)
    print(f"d={d}: n_full={3**d}, number of classes={n_classes}, Avg. req. points for memorization n_avg={avg_mem_size:.2f}, n_full/n_avg={(3**d)/avg_mem_size}")

d=2: n_full=9, number of classes=3, Avg. req. points for memorization n_avg=6.26, n_full/n_avg=1.4378698224852071
d=4: n_full=81, number of classes=3, Avg. req. points for memorization n_avg=55.28, n_full/n_avg=1.4653266331658292
d=8: n_full=6561, number of classes=3, Avg. req. points for memorization n_avg=4373.92, n_full/n_avg=1.5000285785051535


# Question 6.2 a

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_text
data = load_breast_cancer()

In [ ]:
X = data.data
y = data.target  # Convert to binary classification problem

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf_OG = DecisionTreeClassifier(max_depth=None,  # No maximum depth to allow more complexity
                             min_samples_split=2,  # Minimum samples required to split a node
                             min_samples_leaf=1,  # Minimum samples required to be a leaf node
                             random_state=42)

clf_OG.fit(X_train, y_train)

def clauses(data, clf):
  tree_rules = export_text(clf, feature_names=list(data.feature_names))
  predictions = clf.predict(X_test)
  accuracy = accuracy_score(y_test, predictions)
  print(f"Accuracy on the test set: {accuracy:.2f}")
  return tree_rules

def num_clauses(tree_rules):
  if_then_clauses = [line for line in tree_rules.split('\n') if "class:" in line]
  number_of_clauses = len(if_then_clauses)
  print(f"Number of if-then clauses: {number_of_clauses}")

tree_rules = clauses(data, clf_OG)
max_depth = clf_OG.tree_.max_depth

Accuracy on the test set: 0.94


In [ ]:
num_clauses(tree_rules)

Number of if-then clauses: 16


## Increasing min-samples split and min_samples_leaf

In [ ]:
clf_min = DecisionTreeClassifier(max_depth=None,
                             min_samples_split=8,
                             min_samples_leaf=8,
                             random_state=42)

clf_min.fit(X_train, y_train)
tree_rules = clauses(data, clf_min)
num_clauses(tree_rules)

Accuracy on the test set: 0.94
Number of if-then clauses: 8


## Decreasing max_depth

In [ ]:
clf_depth = DecisionTreeClassifier(max_depth=int(max_depth/3),
                             min_samples_split=2,
                             min_samples_leaf=1,
                             random_state=42)

clf_depth.fit(X_train, y_train)
tree_rules = clauses(data, clf_depth)
num_clauses(tree_rules)

Accuracy on the test set: 0.93
Number of if-then clauses: 4


# Question 6.2 b

## Different Dataset #1

In [ ]:
pip install ucimlrepo


In [ ]:
from ucimlrepo import fetch_ucirepo
banknote_authentication = fetch_ucirepo(id=267)
X = banknote_authentication.data.features
y = banknote_authentication.data.targets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
clf_OG.fit(X_train, y_train)
def clauses_changed(data, clf):
  tree_rules = export_text(clf, feature_names=list(data.features))
  predictions = clf.predict(X_test)
  accuracy = accuracy_score(y_test, predictions)
  print(f"Accuracy on the test set: {accuracy:.2f}")
  return tree_rules
tree_rules = clauses_changed(banknote_authentication.data, clf_OG)
num_clauses(tree_rules)

Accuracy on the test set: 0.98
Number of if-then clauses: 27


In [ ]:
clf_min.fit(X_train, y_train)
tree_rules = clauses_changed(banknote_authentication.data, clf_min)
num_clauses(tree_rules)

Accuracy on the test set: 0.97
Number of if-then clauses: 21


In [ ]:
clf_depth.fit(X_train, y_train)
tree_rules = clauses_changed(banknote_authentication.data, clf_depth)
num_clauses(tree_rules)

Accuracy on the test set: 0.91
Number of if-then clauses: 4


## Different Dataset #2

In [ ]:
from ucimlrepo import fetch_ucirepo
haberman_s_survival = fetch_ucirepo(id=43)
X = haberman_s_survival.data.features
y = haberman_s_survival.data.targets
print(haberman_s_survival.metadata)
print(haberman_s_survival.variables)


{'uci_id': 43, 'name': "Haberman's Survival", 'repository_url': 'https://archive.ics.uci.edu/dataset/43/haberman+s+survival', 'data_url': 'https://archive.ics.uci.edu/static/public/43/data.csv', 'abstract': 'Dataset contains cases from study conducted on the survival of patients who had undergone surgery for breast cancer', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 306, 'num_features': 3, 'feature_types': ['Integer'], 'demographics': ['Age'], 'target_col': ['survival_status'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1976, 'last_updated': 'Mon Mar 04 2024', 'dataset_doi': '10.24432/C5XK51', 'creators': ['S. Haberman'], 'intro_paper': None, 'additional_info': {'summary': "The dataset contains cases from a study that was conducted between 1958 and 1970 at the University of Chicago's Billings Hospital on the survival of patients who had undergone surgery

In [ ]:
clf_OG.fit(X_train, y_train)
tree_rules = clauses_changed(banknote_authentication.data, clf_OG)
num_clauses(tree_rules)

Accuracy on the test set: 0.98
Number of if-then clauses: 27


In [ ]:
clf_min.fit(X_train, y_train)
tree_rules = clauses_changed(banknote_authentication.data, clf_min)
num_clauses(tree_rules)

Accuracy on the test set: 0.97
Number of if-then clauses: 21


In [ ]:
clf_depth.fit(X_train, y_train)
tree_rules = clauses_changed(banknote_authentication.data, clf_depth)
num_clauses(tree_rules)

Accuracy on the test set: 0.91
Number of if-then clauses: 4


# Question 6.2 c

In [ ]:
def evaluate_decision_tree(num_samples, num_features, clf):
    X = np.random.randint(2, size=(num_samples, num_features))
    y = np.random.randint(2, size=(num_samples))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    print(f"Accuracy for {num_samples} samples and {num_features} features: {accuracy:.2f}")
classifiers = {
    'clf_OG': clf_OG,
    'clf_min': clf_min,
    'clf_depth': clf_depth
}
# Evaluate the model with different numbers of samples and features
for clf_name, clf in classifiers.items():
  print(clf_name)
  for num_samples in [100, 500, 1000]:
        for num_features in [5, 10, 20]:
            evaluate_decision_tree(num_samples, num_features, clf)

clf_OG
Accuracy for 100 samples and 5 features: 0.43
Accuracy for 100 samples and 10 features: 0.43
Accuracy for 100 samples and 20 features: 0.53
Accuracy for 500 samples and 5 features: 0.47
Accuracy for 500 samples and 10 features: 0.52
Accuracy for 500 samples and 20 features: 0.54
Accuracy for 1000 samples and 5 features: 0.54
Accuracy for 1000 samples and 10 features: 0.45
Accuracy for 1000 samples and 20 features: 0.45
clf_min
Accuracy for 100 samples and 5 features: 0.50
Accuracy for 100 samples and 10 features: 0.60
Accuracy for 100 samples and 20 features: 0.33
Accuracy for 500 samples and 5 features: 0.59
Accuracy for 500 samples and 10 features: 0.47
Accuracy for 500 samples and 20 features: 0.50
Accuracy for 1000 samples and 5 features: 0.52
Accuracy for 1000 samples and 10 features: 0.49
Accuracy for 1000 samples and 20 features: 0.49
clf_depth
Accuracy for 100 samples and 5 features: 0.53
Accuracy for 100 samples and 10 features: 0.63
Accuracy for 100 samples and 20 feat

# Question 6.3 a

In [ ]:
# Generate a long random string of 10000 characters
random_string = ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789', k=10000))

# Compress the string using zlib
compressed_string = zlib.compress(random_string.encode())

# Calculate the compression ratio
compression_ratio = len(compressed_string) / len(random_string)

(len(compressed_string), compression_ratio)


(7526, 0.7526)

# 6.3 b

In the realm of lossless compression, as applied to highly random or uniformly distributed data (like the long random string we generated), the theory suggests that significant compression (i.e., a substantial reduction in size) is typically not achievable. This is because the compression ratio's theoretical best-case scenario, especially for lossless algorithms, hinges on the presence of redundancy within the data. For completely random data, which lacks any pattern or redundancy, the expected compression ratio approaches 1, meaning no effective compression is achieved.

The expected compression ratio (G) in such contexts is guided by the notion that the entropy (a measure of randomness or unpredictability) in highly random data is maximal. When data is maximally random, each bit is as informative as possible, and therefore, there's little to no scope for reducing the amount of information (hence, size) without losing data integrity, which is not permissible in lossless compression.

From Chapter 6, it is likely discussed that the theoretical limit for the compression ratio for a perfectly efficient compression algorithm would be bounded by the entropy per bit in the data. However, since real-world algorithms may not reach this perfect efficiency, especially for random data, we expect the compression ratio to hover around 1, as observed with the random string example.

In the specific case of our experiment with the random string, the observed compression ratio was approximately 0.7528. This is slightly less than 1, indicating a small reduction from the original size, but still close to the expected value for highly random data. The slight reduction might be due to some coincidental patterns or the overhead reduction in specific parts of the data, but it's far from the significant compression seen with more redundant or patterned data.

Therefore, in line with the theory from Chapter 6, the expected compression ratio for lossless compression applied to a dataset with high randomness (like our generated string) would indeed be close to 1, reflecting the minimal efficiency in compressing such data due to its lack of redundancy and high entropy.

# Question 8.4 b

In [ ]:
def memorize(data, labels):
    # Step 1: Pair sum of d-dimensional vectors with labels
    table = [(np.sum(vector), label) for vector, label in zip(data, labels)]

    # Step 2: Sort the table based on the sum
    sorted_table = sorted(table, key=lambda x: x[0])

    # Step 3: Determine the number of thresholds
    thresholds = 0
    previous_label = None
    for _, label in sorted_table:
        if label != previous_label:
            thresholds += 1
            previous_label = label

    # Step 4: Calculate minimum thresholds
    min_thresholds = np.log2(thresholds + 1)

    # Step 5: Calculate MEC using the adjusted formula
    d = data.shape[1]  # Dimensionality of the input vectors
    mec = (min_thresholds * (d + 1)) + min_thresholds

    return mec

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

points = df.iloc[:, :-1].values  # Selects all rows and all columns except the last one as data
labels = df.iloc[:, -1].values

memorize(points, labels)


199.32219809586817

In [ ]:
def memorize_extended(data, labels):
    overall_mec = 0
    num_classifications = labels.shape[1]

    for i in range(num_classifications):
        current_labels = labels[:, i]

        mec = memorize(data, current_labels)
        overall_mec += mec

    return overall_mec

# Example usage with a hypothetical dataset and labels for multiple binary classifications
data = np.array([
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 5]
])

labels = np.array([
    [0, 1],
    [1, 0],
    [0, 1],
    [1, 0]
])

overall_mec = memorize_extended(data, labels)
print(f"Overall Memory-Equivalent Capacity: {overall_mec}")

Overall Memory-Equivalent Capacity: 18.575424759098897


# Question 8.4 c

In [ ]:
def memorize_regression(data, labels, num_bins):
    bins = np.linspace(min(labels), max(labels), num_bins + 1)
    binned_labels = np.digitize(labels, bins) - 1  # Convert to bin indices

    mec = memorize(data, binned_labels)
    return mec, binned_labels

In [ ]:
regression_data = np.array([
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 5]
])
regression_labels = np.array([100, 200, 300, 400])
num_bins = 2  # Example number of bins
mec, binned_labels = memorize_regression(regression_data, regression_labels, num_bins)
print(f"Approximated Memory-Equivalent Capacity for Regression: {mec}")
print(f"Binned Labels: {binned_labels}")

Approximated Memory-Equivalent Capacity for Regression: 8.0
Binned Labels: [0 0 1 2]


# Question 9

In [ ]:
pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the data
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape images for the CNN input
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=64)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

Epoch 1/5
938/938 [==============================] - 68s 70ms/step - loss: 0.1820 - accuracy: 0.9447
Epoch 2/5
938/938 [==============================] - 55s 58ms/step - loss: 0.0513 - accuracy: 0.9836
Epoch 3/5
938/938 [==============================] - 53s 57ms/step - loss: 0.0369 - accuracy: 0.9885
Epoch 4/5
938/938 [==============================] - 53s 57ms/step - loss: 0.0299 - accuracy: 0.9902
Epoch 5/5
313/313 [==============================] - 5s 14ms/step - loss: 0.0338 - accuracy: 0.9904
Test accuracy: 0.9904000163078308


## Adding more convolutional layers and activation functions change

In [ ]:
model_v1 = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_v1.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model_v1.fit(train_images, train_labels, epochs=5, batch_size=64)
test_loss, test_acc = model_v1.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

Epoch 1/5
938/938 [==============================] - 228s 242ms/step - loss: 0.1274 - accuracy: 0.9603
Epoch 2/5
938/938 [==============================] - 224s 239ms/step - loss: 0.0394 - accuracy: 0.9877
Epoch 3/5
938/938 [==============================] - 225s 239ms/step - loss: 0.0269 - accuracy: 0.9912
Epoch 4/5
938/938 [==============================] - 223s 238ms/step - loss: 0.0196 - accuracy: 0.9936
Epoch 5/5
313/313 [==============================] - 11s 36ms/step - loss: 0.0298 - accuracy: 0.9912
Test accuracy: 0.9911999702453613


## Changing Hyperparameters

In [ ]:
model_v2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_v2.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

model_v2.fit(train_images, train_labels, epochs=5, batch_size=64)
test_loss, test_acc = model_v2.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

Epoch 1/5
938/938 [==============================] - 68s 72ms/step - loss: 0.9492 - accuracy: 0.7164
Epoch 2/5
938/938 [==============================] - 61s 65ms/step - loss: 0.2037 - accuracy: 0.9383
Epoch 3/5
938/938 [==============================] - 73s 77ms/step - loss: 0.1338 - accuracy: 0.9599
Epoch 4/5
938/938 [==============================] - 77s 82ms/step - loss: 0.1047 - accuracy: 0.9675
Epoch 5/5
313/313 [==============================] - 4s 13ms/step - loss: 0.0657 - accuracy: 0.9801
Test accuracy: 0.9800999760627747


## Applying Measurements

In [ ]:
hyperparameter_grid = {
    'num_layers': [2, 3, 4],  # Number of convolutional layers
    'num_neurons': [32, 64, 128],  # Number of neurons in the dense layer
    'batch_size': [32, 64, 128],
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [0.001, 0.01]
}

In [ ]:
def train_evaluate_model(num_layers, num_neurons, batch_size, optimizer, learning_rate):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    for _ in range(num_layers - 1):
        model.add(layers.Conv2D(num_neurons, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(num_neurons, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    opt = None
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(train_images, train_labels, epochs=5, batch_size=batch_size, validation_split=0.1, verbose=0)

    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
    return test_acc

In [ ]:
from sklearn.model_selection import ParameterGrid

best_acc = 0
best_params = {}

for params in ParameterGrid(hyperparameter_grid):
    acc = train_evaluate_model(**params)
    if acc > best_acc:
        best_acc = acc
        best_params = params
    print(f"Tested: {params}, Accuracy: {acc}")

print(f"Best Accuracy: {best_acc}, Best Params: {best_params}")

Tested: {'batch_size': 32, 'learning_rate': 0.001, 'num_layers': 2, 'num_neurons': 32, 'optimizer': 'adam'}, Accuracy: 0.9872999787330627
Tested: {'batch_size': 32, 'learning_rate': 0.001, 'num_layers': 2, 'num_neurons': 32, 'optimizer': 'sgd'}, Accuracy: 0.9247000217437744
Tested: {'batch_size': 32, 'learning_rate': 0.001, 'num_layers': 2, 'num_neurons': 64, 'optimizer': 'adam'}, Accuracy: 0.9883000254631042
Tested: {'batch_size': 32, 'learning_rate': 0.001, 'num_layers': 2, 'num_neurons': 64, 'optimizer': 'sgd'}, Accuracy: 0.9290000200271606
Tested: {'batch_size': 32, 'learning_rate': 0.001, 'num_layers': 2, 'num_neurons': 128, 'optimizer': 'adam'}, Accuracy: 0.9894000291824341
Tested: {'batch_size': 32, 'learning_rate': 0.001, 'num_layers': 2, 'num_neurons': 128, 'optimizer': 'sgd'}, Accuracy: 0.929099977016449
Tested: {'batch_size': 32, 'learning_rate': 0.001, 'num_layers': 3, 'num_neurons': 32, 'optimizer': 'adam'}, Accuracy: 0.9825000166893005
Tested: {'batch_size': 32, 'learning

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_51. Consider increasing the input size. Received input shape [None, 1, 1, 32] which would produce output shape with a zero or negative value in a dimension.

### In these experiments, it showcases that it is now the architecture or hyperparameters that determines model behaviour, solely the dataset. Thus, while time to train may vary, all models with the same capacity can learn the same functions.